## Accenture - Supply Chain


In [73]:
# Packaging management
import numpy as np
import pandas as pd
import csv 
from sklearn.linear_model import LinearRegression as lm
import statsmodels

In [72]:
## Probability
#Original
pd.read_csv("datas/orders.csv", index_col=1, header=None, squeeze=True).to_dict()
#Final
pd_cities_raw = pd.read_csv("datas/cities_data.csv" ,sep=";")

city_list_l = pd_cities_raw['late_order']
city_list_f = pd_cities_raw['city_to_name']
cities_f = pd_cities_raw['city_to_name'].unique()
final_order = list(set(city_list_f) | set(city_list_l))



/var/folders/n5/ktlz2xtn5vs65v3vw81n10k80000gn/T/ipykernel_59342/24850140.py:3: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  pd.read_csv("datas/orders.csv", index_col=0, header=None, squeeze=True).to_dict()


KeyError: 'late_order'

In [82]:
## Distance matrix preparation

## Read raw data from csv files
pd_cities_raw = pd.read_csv("datas/cities_data.csv" ,sep=";")

city_list_s = pd_cities_raw['city_from_name'].unique()
city_list_d = pd_cities_raw['city_to_name'].unique()
all_cities= list(set(city_list_s) | set(city_list_d))

city_dict = {all_cities[i]:i for i in range(0,len(all_cities))}
n_cities = len(all_cities)

distance_mat = [[0]*n_cities]*n_cities


for index, row in pd_cities_raw.iterrows():
    s = city_dict[row["city_from_name"]]
    d = city_dict[row["city_to_name"]]
    distance = row["distance"]
    distance_mat[s][d] = distance

In [65]:
## Product Matrix

pd_products_raw = pd.read_csv("datas/product_attributes.csv", sep=",")

##dataframe quering example  !(return a string value)
# pd_products_raw.loc[pd_products_raw["product_id"]==1672707]["material_handling"].to_string(index=False)


3

In [83]:
pd_orders_raw=pd.read_csv("datas/orders.csv",sep=";")


pd_orders_raw.value_counts().reset_index(name="delays")

delay_rate_origin = {all_cities[i]:0 for i in range(0,len(all_cities))}

In [84]:
delay_rate_origin

{'Helsinki': 0,
 'Warsaw': 0,
 'Marseille': 0,
 'Malmö': 0,
 'Valencia': 0,
 'Copenhagen': 0,
 'Berlin': 0,
 'Bremen': 0,
 'Bucharest': 0,
 'Rotterdam': 0,
 'Hanover': 0,
 'Bordeaux': 0,
 'Lisbon': 0,
 'Hamburg': 0,
 'Budapest': 0,
 'Rome': 0,
 'Paris': 0,
 'Amsterdam': 0,
 'Zaragoza': 0,
 'Cologne': 0,
 'Munich': 0,
 'Prague': 0,
 'Athens': 0,
 'Liege': 0,
 'Milan': 0,
 'Madrid': 0,
 'Porto': 0,
 'Vienna': 0,
 'Barcelona': 0,
 'Turin': 0,
 'Bratislava': 0,
 'Lille': 0,
 'Stockholm': 0,
 'Lyon': 0,
 'Naples': 0,
 'Venlo': 0,
 'Dusseldorf': 0}

In [79]:
data

,order_id,origin_port,customs_procedures,logistic_hub,customer,product_id,units,late_order
0,366c7a3d298f,Rotterdam,DTP,Venlo,Marseille,1692723,583,True
1,45f906331e10,Rotterdam,CRF,Rome,Marseille,1644308,459,False
2,ac84a6e4af0f,Athens,CRF,Venlo,Paris,1684170,464,True
3,f5e98cb29790,Rotterdam,CRF,Lille,Milan,1620510,678,False
4,a9e7c9bee35b,Barcelona,CRF,Venlo,Berlin,1699372,353,False
...,...,...,...,...,...,...,...,...
114271,3f4b15fb770e,Rotterdam,CRF,Dusseldorf,Bordeaux,1681376,645,False
114272,d2e6978a38fd,Barcelona,DTD,Dusseldorf,Berlin,1676942,502,False
114273,b88babd5c7bd,Rotterdam,DTP,Dusseldorf,Rome,1692737,464,False
114274,b0b5c761613f,Barcelona,DTD,Dusseldorf,Munich,1699974,388,False
